### Transfer Model

In [2]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.models import efficientnet_b0

# Load pre-trained EfficientNet-B0
model = efficientnet_b0(pretrained=True)

# Modify the classifier for regression (outputting a single scalar)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, 1)

# Optional: Freeze feature extractor layers (at first)
for param in model.features.parameters():
    param.requires_grad = False



/opt/anaconda3/envs/neuralnetworks/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/anaconda3/envs/neuralnetworks/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


### Preprocess

In [3]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

class CrowdDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.images_dir = os.path.join(data_dir, 'images')
        self.label_path = os.path.join(data_dir, 'image_labels.txt')
        self.transform = transform

        self.samples = []
        with open(self.label_path, 'r') as f:
            for line in f:
                parts = line.strip().split(',')
                if len(parts) >= 2:
                    image_id = parts[0]
                    count = float(parts[1])
                    if count > 1000:
                        return  # skip this image
                    filename = image_id + '.jpg'
                    self.samples.append((filename, count))


    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        filename, count = self.samples[idx]
        image_path = os.path.join(self.images_dir, filename)

        image = Image.open(image_path).convert("RGB")
        if self.transform:
            image = self.transform(image)

        count = torch.tensor(count, dtype=torch.float32)

        return image, count
    
transform = transforms.Compose([
    # transforms.Grayscale(num_output_channels=1),
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

# Define paths to each data split
base_dir = './jhu_crowd_v2.0'
splits = {
    'train': os.path.join(base_dir, 'train'),
    'val': os.path.join(base_dir, 'val'),
    'test': os.path.join(base_dir, 'test')
}

### Training Setup

In [4]:
# loss function
criterion = nn.MSELoss()  # or try MAE with nn.L1Loss()

In [5]:
# optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
from torch.utils.data import DataLoader

train_dataset = CrowdDataset(splits['train'], transform=transform)
val_dataset = CrowdDataset(splits['val'], transform=transform)
test_dataset = CrowdDataset(splits['test'], transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

### Forward Pass Loop

In [ ]:
model.eval()  # or model.train() if training
for images, labels in train_loader:
    # images = images.to(device)  # If using CUDA
    # labels = labels.float().to(device)  # Make sure labels are float

    outputs = model(images)  # Shape: [batch_size, 1]
    outputs = outputs.squeeze(1)  # Shape: [batch_size]

    loss = criterion(outputs, labels)

    # If training:
    # optimizer.zero_grad()
    # loss.backward()
    # optimizer.step()